# 맛집 유튜브 채널 찾기
결과 : data/youtube_channel.xlsx  

방법1 : 목록을 정리해 놓은 글을 찾아서 정리한다.  
방법2 : [PLAYBOARD](https://playboard.co/)에서 특정 검색어(부산 맛집)를 입력해 크롤링한다.   
방법3 : 유튜브에서 특정 검색어(맛집)를 입력해 크롤링한다.  

# 유튜브 채널에서 추천한 맛집 찾기
~~방법1 : 유튜브API : 동영상은 제공해주지만 동영상 제목은 제공해주지 않습니다.~~  
방법 : 유튜브 크롤링

In [120]:
# 사용한 라이브러리
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.keys import Keys
import urllib
import numpy as na
import re

In [30]:
# 유튜브 채널 가져오기
youtube_channels = pd.read_excel("./data/youtube_channel.xlsx", sheet_name="Sheet1")
# '제공'열 삭제
youtube_channels.drop('제공', axis=1, inplace=True)
# 중복 데이터 삭제
youtube_channels.drop_duplicates(keep='first')
display(youtube_channels.head(5))

,크리에이터
0,맛상무
1,정육왕
2,맛있겠다 Yummy
3,섬마을훈태TV
4,떡볶퀸 Tteokbokqueen


In [31]:
# 크롤링
# 드라이버 준비
driver = webdriver.Chrome('C:/Temp/chromedriver')
driver.implicitly_wait(3)

In [34]:
# 해당 채널 인기동영상 페이지 요청
# 모든 채널의 url이 저렇게 되지 않습니다.
key = youtube_channels.loc[9, '크리에이터']
url = "https://www.youtube.com/c/" + urllib.parse.quote(key).replace("%20", "") + "/videos?view=0&sort=p&flow=grid"
driver.get(url)
time.sleep(2)

# 츄릅켠 맛집 정보가져오기

In [ ]:
# 츄릅켠 홈 채널 들어가기
driver.get("https://www.youtube.com/c/ekekekTV/videos?view=0&sort=p&flow=grid")
# 30개의 동영상 url 가져오기
temp = []
video_titles = driver.find_elements_by_css_selector("#video-title")
for rank, video_title in enumerate(video_titles, 1):
    temp.append([rank, video_title.text, video_title.get_attribute("href")])

In [ ]:
# 2차원 배열 -> 데이터프레임
df = pd.DataFrame(temp, 
                   columns=['인기도 순위', '동영상 제목', '동영상 링크'])
display(df)

In [ ]:
df["정보"] = na.NaN
df["업로드날짜"] = na.NaN
for i in range(df.shape[0]):
    print(df.loc[i, "동영상 링크"])
    driver.get(df.loc[i, "동영상 링크"])
    time.sleep(5)
    info = driver.find_element_by_css_selector("#description > yt-formatted-string > span:nth-child(1)")
    print(info.text)
    df.loc[i, "정보"] = info.text
    date = driver.find_element_by_css_selector("#info-strings > yt-formatted-string")
    print(date.text)
    df.loc[i, "업로드날짜"] = date.text
    
df["맛집"] = na.NaN
for row in range(df.shape[0]):
    df.loc[row, "맛집"] = re.search("^\[.*", df.loc[row, "정보"]).group()[1:-1]
    
# 채널이름 넣기
df["정보제공"] = "츄릅켠"

# 더들리 맛집 정보가져오기

In [123]:
# 더들리 홈 채널 들어가기
driver.get("https://www.youtube.com/c/%EB%8D%94%EB%93%A4%EB%A6%AC/videos?view=0&sort=p&flow=grid")
# 30개의 동영상 url 가져오기
temp = []
video_titles = driver.find_elements_by_css_selector("#video-title")
for rank, video_title in enumerate(video_titles, 1):
    temp.append([rank, video_title.text, video_title.get_attribute("href")])

In [124]:
# 2차원 배열 -> 데이터프레임
df2 = pd.DataFrame(temp, 
                   columns=['인기도 순위', '동영상 제목', '동영상 링크'])
display(df2)

,인기도 순위,동영상 제목,동영상 링크
0,1,미슐랭 2스타 정식당에 다녀왔습니다 - michelin guide seoul,https://www.youtube.com/watch?v=XDCKmtEAjNo
1,2,호텔 1박하면 무제한 음식을 4번 주는 신라호텔 사육 호캉스 - 이그제큐티브 숙박을...,https://www.youtube.com/watch?v=lmcRXlkhiUc
2,3,랍스터 무한리필 바이킹스워프는 요즘에 어떻게 바뀌었을까?,https://www.youtube.com/watch?v=PuCBB-8Ibwg
3,4,그랜드 워커힐 더뷔페 - 주말 런치 장단점 비교후기,https://www.youtube.com/watch?v=W7xrSj5S2LY
4,5,미슐랭3스타 신라호텔 라연 - 런치코스,https://www.youtube.com/watch?v=egFd72aNfik
5,6,"리뉴얼 된 63빌딩 파빌리온 뷔페의 장단점은?! 1인 115,000원의 저녁 뷔페",https://www.youtube.com/watch?v=nkPvbySFhQg
6,7,서울 조선호텔 뷔페 아리아의 장단점은?,https://www.youtube.com/watch?v=Nh7WuRomPxw
7,8,호텔 1박하면 무제한 음식을 3번 주는 호캉스 - 더 플라자 호텔의 클럽 라운지 숙박,https://www.youtube.com/watch?v=z4ez4-1rrJI
8,9,"스테이크 무한리필! 텍사스데브라질 58,000원 디너는 어떤 고기들이 나올까",https://www.youtube.com/watch?v=wdff2CuUfTI
9,10,서울 신라호텔 뷔페 더 파크뷰의 장단점은?!,https://www.youtube.com/watch?v=I-pXr-0Ju0k


In [155]:
df2["정보"] = na.NaN
df2["업로드날짜"] = na.NaN
df2["맛집"] = na.NaN
for i in range(df.shape[0]):
    print(df2.loc[i, "동영상 링크"])
    driver.get(df2.loc[i, "동영상 링크"])
    time.sleep(5)
    info = driver.find_element_by_css_selector("#description > yt-formatted-string > span:nth-child(3)")
    # print(info.text)
    df2.loc[i, "정보"] = info.text
    date = driver.find_element_by_css_selector("#info-strings > yt-formatted-string")
    print(date.text)
    df2.loc[i, "업로드날짜"] = date.text
    restaurant = driver.find_element_by_css_selector("#container > yt-formatted-string > a:nth-child(1)")
    print(restaurant.text)
    df2.loc[i, "맛집"] = restaurant.text

https://www.youtube.com/watch?v=XDCKmtEAjNo
2020. 3. 23.
#정식당
https://www.youtube.com/watch?v=lmcRXlkhiUc
2021. 3. 5.
#신라호텔
https://www.youtube.com/watch?v=PuCBB-8Ibwg
2020. 9. 24.
#바이킹스워프
https://www.youtube.com/watch?v=W7xrSj5S2LY
2020. 5. 19.
#워커힐호텔
https://www.youtube.com/watch?v=egFd72aNfik
2020. 6. 11.
#신라호텔
https://www.youtube.com/watch?v=nkPvbySFhQg
2021. 8. 12.
#63빌딩
https://www.youtube.com/watch?v=Nh7WuRomPxw
2020. 11. 4.
#조선호텔
https://www.youtube.com/watch?v=z4ez4-1rrJI
2021. 5. 31.
#플라자호텔
https://www.youtube.com/watch?v=wdff2CuUfTI
2020. 12. 4.
#스테이크
https://www.youtube.com/watch?v=I-pXr-0Ju0k
2020. 2. 5.
#신라호텔
https://www.youtube.com/watch?v=iz7cvkthv1Y
2020. 11. 13.
#비채나
https://www.youtube.com/watch?v=vuJDN5izjxE
2021. 10. 7.
#포시즌스
https://www.youtube.com/watch?v=NwHiC48mHmw
2020. 10. 12.
#로우앤슬로우
https://www.youtube.com/watch?v=FgYy1-NmkE8
2020. 9. 10.
#팔선
https://www.youtube.com/watch?v=jx-UHsbPUQw
2020. 8. 30.
#간장게장
https://www.youtube.com/watch?v=QDJ8wnrevmg
2021. 2. 

In [160]:
for row in range(df.shape[0]):
    df2.loc[row, "맛집"] = df2.loc[row, "맛집"].replace("#","")

In [161]:
# 채널이름 넣기
df2["정보제공"] = "더들리"

In [162]:
df2

,인기도 순위,동영상 제목,동영상 링크,정보,업로드날짜,맛집,정보제공
0,1,미슐랭 2스타 정식당에 다녀왔습니다 - michelin guide seoul,https://www.youtube.com/watch?v=XDCKmtEAjNo,* Email : dudely610@gmail.com\n\n-------------...,2020. 3. 23.,정식당,더들리
1,2,호텔 1박하면 무제한 음식을 4번 주는 신라호텔 사육 호캉스 - 이그제큐티브 숙박을...,https://www.youtube.com/watch?v=lmcRXlkhiUc,* Email : dudely610@gmail.com\n\n-------------...,2021. 3. 5.,신라호텔,더들리
2,3,랍스터 무한리필 바이킹스워프는 요즘에 어떻게 바뀌었을까?,https://www.youtube.com/watch?v=PuCBB-8Ibwg,* Email : dudely610@gmail.com\n\n-------------...,2020. 9. 24.,바이킹스워프,더들리
3,4,그랜드 워커힐 더뷔페 - 주말 런치 장단점 비교후기,https://www.youtube.com/watch?v=W7xrSj5S2LY,* Email : dudely610@gmail.com\n\n-------------...,2020. 5. 19.,워커힐호텔,더들리
4,5,미슐랭3스타 신라호텔 라연 - 런치코스,https://www.youtube.com/watch?v=egFd72aNfik,* Email : dudely610@gmail.com\n\n-------------...,2020. 6. 11.,신라호텔,더들리
5,6,"리뉴얼 된 63빌딩 파빌리온 뷔페의 장단점은?! 1인 115,000원의 저녁 뷔페",https://www.youtube.com/watch?v=nkPvbySFhQg,* Email : dudely610@gmail.com\n\n-------------...,2021. 8. 12.,63빌딩,더들리
6,7,서울 조선호텔 뷔페 아리아의 장단점은?,https://www.youtube.com/watch?v=Nh7WuRomPxw,* Email : dudely610@gmail.com\n\n-------------...,2020. 11. 4.,조선호텔,더들리
7,8,호텔 1박하면 무제한 음식을 3번 주는 호캉스 - 더 플라자 호텔의 클럽 라운지 숙박,https://www.youtube.com/watch?v=z4ez4-1rrJI,* Email : dudely610@gmail.com\n\n-------------...,2021. 5. 31.,플라자호텔,더들리
8,9,"스테이크 무한리필! 텍사스데브라질 58,000원 디너는 어떤 고기들이 나올까",https://www.youtube.com/watch?v=wdff2CuUfTI,* Email : dudely610@gmail.com\n\n-------------...,2020. 12. 4.,스테이크,더들리
9,10,서울 신라호텔 뷔페 더 파크뷰의 장단점은?!,https://www.youtube.com/watch?v=I-pXr-0Ju0k,* Email : dudely610@gmail.com\n\n-------------...,2020. 2. 5.,신라호텔,더들리


# 각 채널에서 생성된 데이터프레임 합치기

In [163]:
# 나중에 반복문으로 돌려주세요
youtube_restaurant = pd.DataFrame()
youtube_restaurant = pd.concat([youtube_restaurant, df])
youtube_restaurant = pd.concat([youtube_restaurant, df2])

In [164]:
youtube_restaurant.to_csv("./data/youtube_restaurant.csv",index = False, header=True)
youtube_restaurant

,인기도 순위,동영상 제목,동영상 링크,정보,업로드날짜,맛집,정보제공
0,1,해운대 엘시티 시그니엘 부산 뷔페. 전메뉴 완전정복,https://www.youtube.com/watch?v=wHqJSdPQQzE,[해운대 시그니엘부산 더뷰]\n★ 영업시간 및 주소 추가정보는 여기서 확인하세요 ★...,2020. 8. 10.,해운대 시그니엘부산 더뷰,츄릅켠
1,2,미슐랭에 소개된 유일한 삼겹살집. 츄릅로드,https://www.youtube.com/watch?v=bcD0zbaOCu8,[서울 약수 금돼지식당]\n★ 영업시간 및 주소 추가정보는 여기서 확인하세요 ★\n...,2020. 1. 12.,서울 약수 금돼지식당,츄릅켠
2,3,해운대 그랜드조선 부산 신상 뷔페 아리아 완전 정복 (feat. HBD),https://www.youtube.com/watch?v=UI2SkIHhBI4,[해운대 그랜드조선 부산 뷔페 아리아]\n★ 영업시간 및 주소 추가정보는 여기서 확...,2020. 10. 26.,해운대 그랜드조선 부산 뷔페 아리아,츄릅켠
3,4,부산에 새로생긴 랍스터 무한리필 뷔페 바이킹스워프 기장 동부산점 완전정복,https://www.youtube.com/watch?v=InYKvx7tB6o,[부산 기장군 바이킹스 워프]\n★ 영업시간 및 주소 추가정보는 여기서 확인하세요 ...,2021. 10. 6.,부산 기장군 바이킹스 워프,츄릅켠
4,5,해군대령은 왜 이 중국집에 감사장을 줬을까. 생활의달인 용호동 차이나반점,https://www.youtube.com/watch?v=dEHKZFuRxvI,[부산 용호동 차이나반점]\n★ 영업시간 및 주소 추가정보는 여기서 확인하세요 ★\...,2020. 1. 5.,부산 용호동 차이나반점,츄릅켠
5,6,초..초코파이돈까스요? 부산에서 초돈 발견! 화명동 가쯔렛 츄릅로드 (Busan /...,https://www.youtube.com/watch?v=6cn6x5pk-Wo,[부산 화명동 가쯔렛],2019. 8. 22.,부산 화명동 가쯔렛,츄릅켠
6,7,서면에서 돈까스를 주문했더니 두 메뉴를 더 받았습니다. 츄릅로드 (feat. 해태커...,https://www.youtube.com/watch?v=Gi4C9PhATpQ,[부산 서면 티파니돈까스]\n★ 영업시간 및 주소 추가정보는 여기서 확인하세요 ★\...,2019. 11. 16.,부산 서면 티파니돈까스,츄릅켠
7,8,냉삼 100그램을 2만원 주고는 먹을 수가 없었습니다. 츄릅로드,https://www.youtube.com/watch?v=ud-nqyAy_kw,[부산 광안리 삼오불고기]\n★ 영업시간 및 주소 추가정보는 여기서 확인하세요 ★\...,2020. 2. 20.,부산 광안리 삼오불고기,츄릅켠
8,9,해운대 30년 현지 맛집. 삼겹살 구워먹는 쌈밥 츄릅로드 (Ssambap),https://www.youtube.com/watch?v=FPD7NjOwFII,[부산 해운대 소문난원조쌈밥]\n★ 영업시간 및 주소 추가정보는 여기서 확인하세요 ...,2019. 12. 19.,부산 해운대 소문난원조쌈밥,츄릅켠
9,10,울산 쫀드기집 깨알 리뷰. 길거리음식 츄릅로드 (Korean Street food ...,https://www.youtube.com/watch?v=pHSgHvvxSEk,[울산 성남동 쫀드기집]\n\n씹는 재미. 라면스프 치트키.\n여름엔 슬러쉬가 최고...,2019. 8. 6.,울산 성남동 쫀드기집,츄릅켠


# 실험

In [33]:
# 행찾기
youtube_channels.loc[9, '크리에이터']

'츄릅켠'

In [ ]:
# 스크롤내리기
driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)

In [116]:
# 정규표현식
import re
text = "[부산 서면 티파니돈까스]\n차장 (10분에 700원)\n 승용차부제, 경차, 친환경차 50% 감면"
re.search("^\[.*", text).group()[1:-1]

# 하다가 버린 코드

In [97]:
"""
버린이유 : 바로 인기동영상 페이지로 가서 동영상을 가져올려고 한다. 이 코드는 바로 가지 않는다.
"""
# 해당 채널 페이지 요청
key = youtube_channels.loc[4, '크리에이터']
url = "https://www.youtube.com/c/" + urllib.parse.quote(key).replace("%20", "")
driver.get(url)
time.sleep(2)
# 동영상 페이지로 이동
video_link = driver.find_element_by_css_selector("#tabsContent > tp-yt-paper-tab:nth-child(4) > div")
video_link.click()
# 정렬기준 선택
video_link = driver.find_element_by_css_selector("#icon-label")
video_link.click()

In [ ]:
"""
버린이유 : flex 기술로 인해 드라이버가 모르는 태그는 작동을 시킬 수 없다.
"""
# 정렬기준 선택
video_link = driver.find_element_by_css_selector("#menu > a.yt-simple-endpoint.style-scope.yt-dropdown-menu.iron-selected > tp-yt-paper-item")
video_link.click()
time.sleep(1)

In [ ]:
"""
사용한 이유 : &기호를 잘못 변환해서 사용했다.
버린이유 :  왜 인지 인기동영상으로 넘어가지 않는다.
"""
# 해당 채널 페이지 요청
key = youtube_channels.loc[4, '크리에이터']
url = "https://www.youtube.com/c/" + urllib.parse.quote(key).replace("%20", "") + "/videos?view=0&sort=p&flow=grid"
driver.get(url)
time.sleep(2)

In [32]:
"""
버린이유 : 한번에 채널을 불러오는 것이 더 좋을 것이라고 판단해서
"""
# 크롤링
# 드라이버 준비
driver = webdriver.Chrome('C:/Temp/chromedriver')
driver.implicitly_wait(3)
# 키워드를 포함한 페이지 요청 
key = youtube_channels.loc[4, '크리에이터']
url = "https://www.youtube.com/results?search_query=" + urllib.parse.quote(key)
driver.get(url)
time.sleep(2)

In [26]:
"""
버린이유 : 검색창에 글을 못 써서
"""
# 크롤링
# 드라이버 준비
driver = webdriver.Chrome('C:/Temp/chromedriver')
driver.implicitly_wait(3)
# 유튜브 들어가기
driver.get("https://www.youtube.com/")
time.sleep(2)
# 검색어 입력
inputId = driver.find_element_by_id('search')
inputId.send_keys("맛상무")
inputId.send_keys(Keys.ENTER)

In [ ]:
"""
제공 : https://gist.github.com/WoongheeLee/70cb07f2fc1f32a76c82a99d84d51044
버린이유 : 서브 버튼 클릭이 아니라서
"""
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_image_title(url):
    # 웹 드라이버 초기화
    driver_path = 'C:/Temp/chromedriver'
    driver = webdriver.Chrome(driver_path)
    driver.implicitly_wait(5) # or bigger second
    
    # 열고자 하는 채널 -> 동영상 목록으로 된 url 페이지를 엶
    driver.get(url)
    
    image_list = list() # 썸네일을 받을 수 있는 주소 저장용 리스트
    title_list = list() # 썸네일 제목을 저장하는 리스트

    idx = 1
    while True:
        try:
            img_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/ytd-thumbnail/a/yt-img-shadow/img'
            title_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/h3/a'
            
            # 이미지가 곧바로 로드 되지 않을 때, 20초간 강제로 기다림
            img = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, img_xpath)))
            if img is None:
                print(idx, 'img is not loaded.')
            
            # 한 페이지에 약 8개 불러오는 데, 동영상 목록을 추가 불러오기 위해 스크롤 내림
            if idx % 8 == 0 :
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
            
            # 썸네일 주소를 리스트에 저장
            image = driver.find_element_by_xpath(img_xpath)
            img_url = image.get_attribute('src')
            image_list.append(img_url)

            # 타이틀을 리스트에 저장
            title = driver.find_element_by_xpath(title_xpath)
            print(idx,title.text,img_url)
            title_list.append(title.text)

            idx += 1
        except Exception as e:
            print()
            print(e)
            break
    assert len(image_list) == len(title_list)
    driver.close()
    return image_list, title_list

# 자이언트 펭TV
url1 = 'https://m.youtube.com/channel/UCtckgmUcpzqGnzcs7xEqMzQ/videos'
image1, title1 = get_image_title(url1)
print(title1)

# 이슈


해결방안 : https://www.python2.net/questions-895942.htm